In [ ]:
#Problem A
from datetime import datetime, timedelta

def parse(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def format_date(date):
    return date.strftime("%Y-%m-%d")

def days_between(start, end):
    current = start
    while current <= end:
        yield current
        current += timedelta(days=1)

def find_earliest_application(n, y, d, periods):
    last_exit = max(end for _, end in periods)
    out_days = set()
    for start, end in periods:
        out_days.update(days_between(start, end))
    check_date = last_exit + timedelta(days=1)
    while True:
        meets_criteria = True

        for i in range(y):
            year_start = check_date - timedelta(days=(i + 1) * 365)
            year_end = check_date - timedelta(days=i * 365 + 1)

            in_days = sum(1 for day in days_between(year_start, year_end) if day not in out_days)
            if in_days < d:
                meets_criteria = False
                break

        if meets_criteria:
            return format_date(check_date)

        check_date += timedelta(days=1)

In [ ]:
# Problem B
def is_valid(x, y):
    return 0 <= x < 10 and 0 <= y < 10

def dfs(grid, visited, x, y):
    if not is_valid(x, y) or visited[x][y] or grid[x][y] == 1:
        return 0, False

    visited[x][y] = True
    size = 1
    touches_edge = (x == 0 or x == 9 or y == 0 or y == 9)

    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        s, edge = dfs(grid, visited, nx, ny)
        size += s
        touches_edge = touches_edge or edge

    return size, touches_edge

def find_holes(grid):
    visited = [[False]*10 for _ in range(10)]
    holes = []

    for i in range(10):
        for j in range(10):
            if grid[i][j] == 0 and not visited[i][j]:
                size, edge_touch = dfs(grid, visited, i, j)
                if not edge_touch:
                    holes.append(size)

    return holes

In [ ]:
# Problem C
code_map = {
    (1, 1, 1, 1, 2): '0',
    (2, 1, 1, 1, 2): '1',
    (1, 2, 1, 1, 2): '2',
    (2, 2, 1, 1, 1): '3',
    (1, 1, 2, 1, 2): '4',
    (2, 1, 2, 1, 1): '5',
    (1, 2, 2, 1, 1): '6',
    (1, 1, 1, 2, 2): '7',
    (2, 1, 1, 2, 1): '8',
    (1, 2, 1, 2, 1): '9',
    (1, 1, 2, 2, 1): '-',
    (1, 1, 2, 1, 2): 'S'  # Start/Stop
}

weights = {str(i): i for i in range(10)}
weights['-'] = 10

def decode_barcode(widths):
    min_width = min(widths)
    tolerance = 0.05 * min_width

    def is_narrow(w): return abs(w - min_width) <= tolerance
    def is_wide(w): return abs(w - 2 * min_width) <= 2 * tolerance

    def classify_width(w):
        if is_narrow(w): return 1
        if is_wide(w): return 2
        return 0

    pattern = [classify_width(w) for w in widths]
    if 0 in pattern:
        return "bad code"

    symbols = []
    i = 0
    while i + 4 < len(pattern):
        group = tuple(pattern[i:i + 5])
        char = code_map.get(group)

        if not char:
            return "bad code"

        symbols.append(char)
        i += 6 if i + 6 < len(pattern) else 5

    if len(symbols) < 3 or symbols[0] != 'S' or symbols[-1] != 'S':
        return "bad code"

    payload = symbols[1:-1]
    if len(payload) < 2:
        return "bad code"

    data = payload[:-2]
    c_check = payload[-2]
    k_check = payload[-1]

    def compute_check(values, max_weight):
        total = sum((i % max_weight + 1) * weights[ch] for i, ch in enumerate(reversed(values)))
        return total % 11

    if weights[c_check] != compute_check(data, 10):
        return "bad C"

    if weights[k_check] != compute_check(data + [c_check], 9):
        return "bad K"

    return ''.join(data)

case_num = 1

while True:
    try:
        parts = list(map(int, input().split()))
        if parts[0] == 0:
            break

        result = decode_barcode(parts[1:])
        print(f"Case {case_num}: {result}")
        case_num += 1

    except EOFError:
        break